In [26]:
from elasticsearch import Elasticsearch
import pandas as pd

In [27]:
#Navigate to Elasticsearch connect to local host
es = Elasticsearch("http://localhost:9200")
es.info().body

{'name': 'a10f6125f725',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'wnHXhwrkRtygK8fT8jnjfA',
 'version': {'number': '8.7.0',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': '09520b59b6bc1057340b55750186466ea715e30e',
  'build_date': '2023-03-27T16:31:09.816451435Z',
  'build_snapshot': False,
  'lucene_version': '9.5.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [28]:
#Read data file and place into pandas dataframe
df = (
    pd.read_csv("/home/namorak/scripps-research-test/Data/sample_interaction_data.tsv",sep="\t",lineterminator='\n')
    .dropna()
    # .sample(5000, random_state=42)
    .reset_index()
)
#pd.set_option('display.max_colwidth', None)
print(df.head())

   index gene_name gene_claim_name  entrez_id interaction_claim_source  \
0   1257      ROS1            ROS1     6098.0                      DTC   
1   3835      MYLK            MYLK     4638.0                      DTC   
2   4248    CSNK1E          CSNK1E     1454.0                      DTC   
3   4308     EPHA7           EPHA7     2045.0                      DTC   
4   5068     STK11           STK11     6794.0                      DTC   

  interaction_types drug_claim_name drug_claim_primary_name     drug_name  \
0         inhibitor      HESPERADIN              HESPERADIN    HESPERADIN   
1         inhibitor      TOZASERTIB              TOZASERTIB    TOZASERTIB   
2         inhibitor      HTS-466284              HTS-466284  CHEMBL261454   
3         inhibitor      HESPERADIN              HESPERADIN    HESPERADIN   
4         inhibitor      TOZASERTIB              TOZASERTIB    TOZASERTIB   

       drug_concept_id  interaction_group_score     PMIDs  
0  chembl:CHEMBL514409          

In [29]:
#Create index and mappings
mappings = {
    "properties": {
        "index": {"type": "long "},
        "gene_name": {"type": "text"}, 
        "gene_claim_name": {"type": "text"},
        "entrez_id": {"type": "long "},
        "interaction_claim_source": {"type": "text"},
        "interaction_types": {"type": "text"},
        "drug_claim_primary_name": {"type": "text"},    
        "drug_name": {"type": "text"},      
        "drug_concept_id": {"type": "text"},
        "interaction_group_score": {"type": "long "},    
        "PMIDs": {"type": "long "}
    }
}

#es.indices.create(index="medications", mappings=mappings)

In [30]:
#Add data to index
for i, row in df.iterrows():
    doc = {
        "index": row["index"],
        "gene_name": row["gene_name"], 
        "gene_claim_name": row["gene_claim_name"],
        "entrez_id": row["entrez_id"],
        "interaction_claim_source": row["interaction_claim_source"],
        "interaction_types": row["interaction_types"],
        "drug_claim_primary_name": row["drug_claim_primary_name"],    
        "drug_name": row["drug_name"],      
        "drug_concept_id": row["drug_concept_id"],
        "interaction_group_score": row["interaction_group_score"],    
        "PMIDs": row["PMIDs"]
    }
            
es.index(index="medications", id=i, document=doc)

ObjectApiResponse({'_index': 'medications', '_id': '93', '_version': 3, 'result': 'updated', '_shards': {'total': 2, 'successful': 1, 'failed': 0}, '_seq_no': 2, '_primary_term': 1})

In [31]:
#Store bulk data in list
bulk_data = []
for i,row in df.iterrows():
    bulk_data.append(
        {
            "_index": "medications",
            "_id": i,
            "_source": {        
                "index": row["index"],
                "gene_name": row["gene_name"], 
                "gene_claim_name": row["gene_claim_name"],
                "entrez_id": row["entrez_id"],
                "interaction_claim_source": row["interaction_claim_source"],
                "interaction_types": row["interaction_types"],
                "drug_claim_primary_name": row["drug_claim_primary_name"],    
                "drug_name": row["drug_name"],      
                "drug_concept_id": row["drug_concept_id"],
                "interaction_group_score": row["interaction_group_score"],    
                "PMIDs": row["PMIDs"],
            }
        }
    )
bulk(es, bulk_data)


es.indices.refresh(index="medications")
es.cat.count(index="medications", format="json")

NameError: name 'bulk' is not defined

In [ ]:
resp = es.search(
    index="medications",
    query={
            "bool": {
                "must": {
                    "match_phrase": {
                        "gene_name": "CDK7",
                    }
                },
                "filter": {"bool": {"must_not": {"match_phrase": {"column name": "data field"}}}},
            },
        },            
)
resp.body

{'took': 6,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}